### Import necessary packages

In [102]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import librosa
import scipy.io.wavfile as wav
import scipy

from sklearn.model_selection import train_test_split

import os

##### Install dependencies and unzip dataset 

In [2]:
!pip install py7zr
!py7zr x '../input/tensorflow-speech-recognition-challenge/train.7z'
!py7zr x '../input/tensorflow-speech-recognition-challenge/test.7z'

     |████████████████████████████████| 66 kB 2.5 MB/s eta 0:00:011
     |████████████████████████████████| 124 kB 7.3 MB/s eta 0:00:01
     |███████████████████████████████▉| 2.2 MB 7.4 MB/s eta 0:00:01     |████████████████████████████████| 2.2 MB 7.4 MB/s 
     |████████████████████████████████| 1.9 MB 2.9 MB/s eta 0:00:01


### Create train-dataset


In [3]:
TRAIN_DIR = './train/audio'
labels = os.listdir(TRAIN_DIR)

Dataset of all labels except silence

In [174]:
X_mfcc = []
X_chroma = []
Y = []
for label in labels:
    print(label, end=", ")
    path = os.path.join(TRAIN_DIR, label)
    filenames = os.listdir(path)
    for filename in filenames:
        if ".wav" not in filename or label == "_background_noise_":
            continue
        sr, audio = wav.read(os.path.join(path, filename))
        audio = audio.astype('float')
        a = librosa.feature.mfcc(y = audio, sr=sr, n_mfcc=32) # Generate MFCC of the audio sample
        a = np.pad(a, ((0, 0), (0, 32-a.shape[1])), mode="constant", constant_values=0)
        X_mfcc.append(a)
        a = librosa.feature.chroma_stft(y = audio, sr=sr, n_chroma=32) # Generate Chroma STFT of the audio sample
        a = np.pad(a, ((0, 0), (0, 32-a.shape[1])), mode="constant", constant_values=0)
        X_chroma.append(a)
        Y.append(labels.index(label))

five, 

/opt/conda/lib/python3.7/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


dog, _background_noise_, marvin, happy, seven, go, three, four, bird, bed, cat, tree, yes, zero, on, eight, right, stop, wow, sheila, off, nine, left, down, up, one, no, house, two, six, 

Dataset creation of silence label

In [176]:
label = "_background_noise_"
path = os.path.join(TRAIN_DIR, label)
filenames = os.listdir(path)
for filename in filenames:
    if ".wav" not in filename:
        continue
    sr, audio = wav.read(os.path.join(path, filename))
    audio = audio.astype('float')
    for i in range(0, len(audio), sr):
        aa = audio[i:i+sr:10]
        a = librosa.feature.mfcc(y = aa, sr=sr, n_mfcc=32)
        a = np.pad(a, ((0, 0), (0, 32-a.shape[1])), mode="constant", constant_values=0)
        X_mfcc.append(a)
        a = librosa.feature.chroma_stft(y = aa, sr=sr, n_chroma=32)
        a = np.pad(a, ((0, 0), (0, 32-a.shape[1])), mode="constant", constant_values=0)
        X_chroma.append(a)
        Y.append(labels.index(label))

/opt/conda/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1600
  n_fft, y.shape[-1]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: WavFileWarning: Chunk (non-data) not understood, skipping it.
  
/opt/conda/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=249
  n_fft, y.shape[-1]
/opt/conda/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=407
  n_fft, y.shape[-1]
/opt/conda/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1290
  n_fft, y.shape[-1]
/opt/conda/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=293
  n_fft, y.shape[-1]


To balance the dataset add more samples of audio samples for <strong><i>silence</i></strong>. This is done by reversing each audio sample of 1 sec adn generating the said features

In [177]:
label = "_background_noise_"
path = os.path.join(TRAIN_DIR, label)
filenames = os.listdir(path)
for filename in filenames:
    if ".wav" not in filename:
        continue
    sr, audio = wav.read(os.path.join(path, filename))
    audio = audio.astype('float')
    for i in range(0, len(audio), sr):
        aa = audio[i:i+sr:-10]
        a = librosa.feature.mfcc(y = aa, sr=sr, n_mfcc=32)
        a = np.pad(a, ((0, 0), (0, 32-a.shape[1])), mode="constant", constant_values=0)
        X_mfcc.append(a)
        a = librosa.feature.chroma_stft(y = aa, sr=sr, n_chroma=32)
        a = np.pad(a, ((0, 0), (0, 32-a.shape[1])), mode="constant", constant_values=0)
        X_chroma.append(a)
        Y.append(labels.index(label))

/opt/conda/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=0
  n_fft, y.shape[-1]


ValueError: can't extend empty axis 0 using modes other than 'constant' or 'empty'

In [181]:
X_mfcc = np.asarray(X_mfcc)
X_mfcc = np.expand_dims(X_mfcc, -1)
X_chroma = np.asarray(X_chroma)
X_chroma = np.expand_dims(X_chroma, -1)
Y = np.asarray(Y)
X_mfcc.shape, X_chroma.shape, Y.shape

((65123, 32, 32, 1), (65123, 32, 32, 1), (65123,))

To visualize the number of samples for each label

In [182]:
i = np.bincount(Y)
ii = np.nonzero(i)[0]
np.vstack([ii, i[ii]]).T

array([[   0, 2357],
       [   1, 1746],
       [   2,  402],
       [   3, 1746],
       [   4, 1742],
       [   5, 2377],
       [   6, 2372],
       [   7, 2356],
       [   8, 2372],
       [   9, 1731],
       [  10, 1713],
       [  11, 1733],
       [  12, 1733],
       [  13, 2377],
       [  14, 2376],
       [  15, 2367],
       [  16, 2352],
       [  17, 2367],
       [  18, 2380],
       [  19, 1745],
       [  20, 1734],
       [  21, 2357],
       [  22, 2364],
       [  23, 2353],
       [  24, 2359],
       [  25, 2375],
       [  26, 2370],
       [  27, 2375],
       [  28, 1750],
       [  29, 2373],
       [  30, 2369]])

### Train-Test Split

In [183]:
m_train, m_test, c_train, c_test, y_train, y_test = train_test_split(X_mfcc, X_chroma, Y, test_size=0.25, random_state=42)

### Model Creation & Training

In [184]:
def Model(X1, X2, dropout=0.4):
    _input1 = tf.keras.layers.Input(X1)
    x = tf.keras.layers.Conv2D(64, 3, padding="same")(_input1)
    x = tf.keras.layers.Activation(tf.nn.leaky_relu)(x)
    x = tf.keras.layers.BatchNormalization()(x)    
    x = tf.keras.layers.MaxPool2D()(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    
    x = tf.keras.layers.Conv2D(128, 3, padding="same")(x)
    x = tf.keras.layers.Activation(tf.nn.leaky_relu)(x)
    x = tf.keras.layers.BatchNormalization()(x)    
    x = tf.keras.layers.MaxPool2D()(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    
    x = tf.keras.layers.Conv2D(256, 3, padding="same")(x)
    x = tf.keras.layers.Activation(tf.nn.leaky_relu)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPool2D()(x)
    
    x1 = tf.keras.layers.Flatten()(x)
    
    _input2 = tf.keras.layers.Input(X2)
    x = tf.keras.layers.Conv2D(64, 3, padding="same")(_input2)
    x = tf.keras.layers.Activation(tf.nn.leaky_relu)(x)
    x = tf.keras.layers.BatchNormalization()(x)    
    x = tf.keras.layers.MaxPool2D()(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    
    x = tf.keras.layers.Conv2D(128, 3, padding="same")(x)
    x = tf.keras.layers.Activation(tf.nn.leaky_relu)(x)
    x = tf.keras.layers.BatchNormalization()(x)    
    x = tf.keras.layers.MaxPool2D()(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    
    x = tf.keras.layers.Conv2D(256, 3, padding="same")(x)
    x = tf.keras.layers.Activation(tf.nn.leaky_relu)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPool2D()(x)
    
    x2 = tf.keras.layers.Flatten()(x)
    
    x = tf.keras.layers.Concatenate()([x1, x2])
    
    x = tf.keras.layers.Dense(128)(x)
    x = tf.keras.layers.Activation(tf.nn.leaky_relu)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    
    x = tf.keras.layers.Dense(64)(x)
    x = tf.keras.layers.Activation(tf.nn.leaky_relu)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    
    x = tf.keras.layers.Dense(len(labels))(x)
    x = tf.keras.layers.Activation(tf.nn.softmax)(x)
    return tf.keras.Model([_input1, _input2], x)

In [185]:
model = Model(m_train[0].shape, c_train[0].shape)
model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"])
checkpoint_filepath = "weights_mfcc_Conv2D/"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

Summary of the output shape and parameters of the model

In [186]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 32, 32, 1)]  0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 32, 32, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   640         input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 32, 32, 64)   640         input_11[0][0]                   
____________________________________________________________________________________________

In [187]:
H = model.fit([m_train, c_train], y_train, batch_size=128, epochs=500, validation_split=0.25, shuffle=True, callbacks=[model_checkpoint_callback])

Epoch 1/500
287/287 [==============================] - 9s 22ms/step - loss: 3.1394 - accuracy: 0.1800 - val_loss: 3.1489 - val_accuracy: 0.3227
Epoch 2/500
287/287 [==============================] - 6s 20ms/step - loss: 1.3590 - accuracy: 0.6015 - val_loss: 1.3351 - val_accuracy: 0.6083
Epoch 3/500
287/287 [==============================] - 6s 19ms/step - loss: 0.8730 - accuracy: 0.7430 - val_loss: 0.5119 - val_accuracy: 0.8469
Epoch 4/500
287/287 [==============================] - 6s 20ms/step - loss: 0.6679 - accuracy: 0.8024 - val_loss: 0.5604 - val_accuracy: 0.8248
Epoch 5/500
287/287 [==============================] - 6s 20ms/step - loss: 0.5628 - accuracy: 0.8392 - val_loss: 0.3850 - val_accuracy: 0.8851
Epoch 6/500
287/287 [==============================] - 6s 20ms/step - loss: 0.4750 - accuracy: 0.8571 - val_loss: 0.3400 - val_accuracy: 0.8989
Epoch 7/500
287/287 [==============================] - 6s 20ms/step - loss: 0.4381 - accuracy: 0.8675 - val_loss: 0.3063 - val_accuracy:

KeyboardInterrupt: 

Plot of the train history (X-axis: <i>EPOCHS</i>, Y-axis: <i>ACCURACY</i>)

In [ ]:
plt.plot(H.history["accuracy"])
plt.plot(H.history["val_accuracy"])

Load the best weights and evaluate the model

In [188]:
model.load_weights("weights_mfcc_Conv2D/")
print("Train Accuracy", model.evaluate([m_train, c_train], y_train)[1])
print("Test Accuracy", model.evaluate([m_test, c_test], y_test)[1])

1527/1527 [==============================] - 5s 3ms/step - loss: 0.0751 - accuracy: 0.9853
Train Accuracy 0.9852790832519531
509/509 [==============================] - 2s 4ms/step - loss: 0.3277 - accuracy: 0.9384
Test Accuracy 0.9383944272994995


Save the model weights in a zip file (for downloading from server)

In [190]:
!zip -r 'weights_mfcc_Conv2D.zip' 'weights_mfcc_Conv2D/'

updating: weights_mfcc_Conv2D/ (stored 0%)
updating: weights_mfcc_Conv2D/.index (deflated 76%)
updating: weights_mfcc_Conv2D/checkpoint (deflated 34%)
updating: weights_mfcc_Conv2D/.data-00000-of-00001 (deflated 8%)


### Create Test-Dataset


In [191]:
TEST_DIR = './test/audio'

In [196]:
filenames = os.listdir(TEST_DIR)
M_Test = []
C_Test = []
for filename in filenames:
    if ".wav" not in filename:
        continue
    sr, audio = wav.read(os.path.join(TEST_DIR, filename))
    audio = audio.astype('float')
    a = librosa.feature.mfcc(y = audio, sr=sr, n_mfcc=32)
    a = np.pad(a, ((0, 0), (0, 32-a.shape[1])), mode="constant", constant_values=0)
    M_Test.append(a)
    a = librosa.feature.chroma_stft(y = audio, sr=sr, n_chroma=32)
    a = np.pad(a, ((0, 0), (0, 32-a.shape[1])), mode="constant", constant_values=0)
    C_Test.append(a)

In [197]:
M_Test = np.asarray(M_Test)
M_Test = np.expand_dims(M_Test, -1)

C_Test = np.asarray(C_Test)
C_Test = np.expand_dims(C_Test, -1)
M_Test.shape, C_Test.shape

((158538, 32, 32, 1), (158538, 32, 32, 1))

### Load weights and predict class labels for test set provided for the challenge

In [ ]:
checkpoint_filepath = "weights_mfcc_Conv2D/"
model = Model(X_Test[0].shape)
model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.load_weights(checkpoint_filepath)

In [198]:
pred = model.predict([M_Test, C_Test])

##### Create csv file for the predicted labels in the format specified for submission

In [200]:
_fname = []
_labels = []
ans_labels = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "silence", "unknown"]
for i in range(len(M_Test)):
    y = np.argmax(pred[i])
    y = labels[y]
    if y not in ans_labels:
        if y == "_background_noise_":
            y = "silence"
        else:
            y = "unknown"
    _fname.append(filenames[i])
    _labels.append(y)

In [201]:
df = pd.DataFrame.from_dict({'fname':_fname, 'label':_labels})
df.head()

,fname,label
0,clip_6ffc4eb63.wav,unknown
1,clip_6da59a15b.wav,up
2,clip_c12a2e8fc.wav,yes
3,clip_05b92927e.wav,go
4,clip_e605c025d.wav,off


Save dataframe as csv

In [202]:
df.to_csv('submission.csv', index=False)